In [1]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/game-of-thrones-dataset-text-generation/got1.txt


In [2]:
def read_file(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

data = read_file('/kaggle/input/game-of-thrones-dataset-text-generation/got1.txt')